In [ ]:
# Sustainable Campus Automation

# 1. Install dependencies
!pip install ultralytics opencv-python torch torchvision pandas fpdf requests --quiet

# 2. Imports
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from collections import deque
import csv
from datetime import datetime
import pandas as pd
from fpdf import FPDF
from IPython.display import clear_output, display
from base64 import b64decode
import requests
import os

# 3. YOLOv8 Detector (replace with your trained model)

detector = YOLO("yolov8n.pt")

# 4. Tracker Class

class SimpleTracker:
    def __init__(self, max_len=10):
        self.history = deque(maxlen=max_len)
    def update(self, detections):
        self.history.append(detections)
        return self.history

# 5. Action Classification, Energy, Points & Recommendations

DEVICE_POWER = {"AC":1500, "Light":20, "Projector":300, "Computer":200}

def estimate_energy(device, duration_hours=1):
    return DEVICE_POWER.get(device,0)*duration_hours/1000

def calculate_points(action, energy):
    if action=="sustainable": return int(energy*10)
    elif action=="unsustainable": return -int(energy*5)
    return 0

def classify_action(confidence, device_class, duration_hours=1):
    energy = estimate_energy(device_class, duration_hours)
    if device_class=="AC" and energy>1.0:
        return "unsustainable"
    elif device_class=="Light" and energy<0.05:
        return "sustainable"
    elif device_class=="Projector" and energy<0.3:
        return "sustainable"
    elif device_class=="Computer" and energy<0.2:
        return "sustainable"
    else:
        return "neutral"

def recommend(device):
    alternatives = {
        "AC":"Inverter AC (5-star, VRF)",
        "Light":"Smart LED with motion sensor",
        "Projector":"Energy-efficient LED projector",
        "Computer":"Energy Star certified mini PC"
    }
    return alternatives.get(device,"No recommendation")

# 6. Classroom Setup

classrooms = ["Classroom 1","Classroom 2"]  # extendable
trackers = {room: SimpleTracker() for room in classrooms}

BASE_DIR = "./SustainableCampusOutputs"
os.makedirs(BASE_DIR, exist_ok=True)

log_files = {room: os.path.join(BASE_DIR,f"{room.replace(' ','_')}_events.csv") for room in classrooms}
video_files = {room: os.path.join(BASE_DIR,f"{room.replace(' ','_')}_output.mp4") for room in classrooms}
video_writers = {}

# Initialize CSV logs
for room, log in log_files.items():
    with open(log,"w",newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Timestamp","Device","Action","Energy_kWh","Points","Recommendation","Alert"])

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# 7. Webcam Capture Helper

def take_webcam_frame(cam_id=0):
    cap = cv2.VideoCapture(cam_id)
    ret, frame = cap.read()
    cap.release()
    if ret:
        return frame
    return None

# 8. PDF Report Generator

def generate_pdf_report(room):
    df = pd.read_csv(log_files[room])
    if df.empty:
        print(f"No data for {room} to generate report.")
        return
    summary = df.groupby(['Device','Action']).agg({'Energy_kWh':'sum','Points':'sum'}).reset_index()

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, f"Sustainability Report - {room}", ln=True, align='C')
    pdf.ln(5)
    pdf.set_font("Arial", '', 12)

    for idx,row in summary.iterrows():
        device = row['Device']
        action = row['Action']
        energy = row['Energy_kWh']
        points = row['Points']
        suggestion = recommend(device)
        alert_status = "ALERT!" if row['Action']=="unsustainable" else ""
        pdf.multi_cell(0, 8, f"Device: {device} | Action: {action} | Energy (kWh): {energy:.2f} | Points: {points} | Suggestion: {suggestion} {alert_status}")
        pdf.ln(1)

    report_file = os.path.join(BASE_DIR,f"{room.replace(' ','_')}_report.pdf")
    pdf.output(report_file)
    print(f"PDF report saved: {report_file}")

# 9. Alerts & Optional Cloud

ENERGY_THRESHOLD_KWH = 1.0
CLOUD_ENDPOINT = None  # replace with your endpoint if needed

def send_alert(room, device, action, energy):
    print(f"[ALERT] {room}: {device} {action} action, energy {energy:.2f} kWh")
    if CLOUD_ENDPOINT:
        try:
            requests.post(CLOUD_ENDPOINT, json={"room":room,"device":device,"action":action,"energy":energy,"timestamp":str(datetime.now())}, timeout=5)
        except:
            pass

# 10. Real-Time Multi-Classroom Capture

print("Multi-classroom real-time capture started. Interrupt with Ctrl+C to stop.")

try:
    while True:
        for room in classrooms:
            frame = take_webcam_frame()
            if frame is None:
                continue

            if room not in video_writers:
                h, w = frame.shape[:2]
                video_writers[room] = cv2.VideoWriter(video_files[room], fourcc, 5, (w,h))

            results = detector(frame, conf=0.4)[0]

            boxes = results.boxes.xyxy.cpu().numpy()
            cls_ids = results.boxes.cls.cpu().numpy()
            confs = results.boxes.conf.cpu().numpy()

            for box, cls_id, conf in zip(boxes, cls_ids, confs):
                cls_name = results.names[int(cls_id)]
                x1, y1, x2, y2 = map(int, box)

                trackers[room].update([cls_name])
                action = classify_action(float(conf), cls_name)
                energy = estimate_energy(cls_name)
                points = calculate_points(action, energy)
                suggestion = recommend(cls_name)
                alert = ""

                if action=="unsustainable" and energy>=ENERGY_THRESHOLD_KWH:
                    alert="ALERT"
                    send_alert(room, cls_name, action, energy)

                color = (0,255,0) if action=="sustainable" else (0,0,255)
                cv2.rectangle(frame,(x1,y1),(x2,y2),color,2)
                cv2.putText(frame,f"{cls_name}|{action}",(x1,y1-10),cv2.FONT_HERSHEY_SIMPLEX,0.6,color,2)

                with open(log_files[room],"a",newline="") as f:
                    writer = csv.writer(f)
                    writer.writerow([datetime.now().strftime("%Y-%m-%d %H:%M:%S"), cls_name, action, energy, points, suggestion, alert])

            video_writers[room].write(frame)

            clear_output(wait=True)
            print(f"--- {room} Dashboard ---")
            df = pd.read_csv(log_files[room])
            display(df.groupby(['Device','Action']).agg({'Energy_kWh':'sum','Points':'sum'}).reset_index())

except KeyboardInterrupt:
    print("Capture stopped by user.")

# Release video writers
for writer in video_writers.values():
    writer.release()

# Generate PDF reports
for room in classrooms:
    generate_pdf_report(room)

print(f"All outputs saved locally in {BASE_DIR}")


Multi-classroom real-time capture started. Interrupt with Ctrl+C to stop.
Capture stopped by user.
No data for Classroom 1 to generate report.
No data for Classroom 2 to generate report.
All outputs saved locally in ./SustainableCampusOutputs
